 #### ДЗ
1. Разделить дата сет на трейн и тест в отношение 50:50 70:30 80:20 (с перемешиванием)
2. Обучать наши модели на трейне. Предсказывать и замерять метрику R^2 и на трейне и на тесте
3. Проверить следующие модели, для каждого разделения:
    а) sales ~ log_tv + radio
    б) sales ~ TV + radio
    в) sales ~ TV + radio + newspaper

In [27]:
# Common imports
import pandas as pd
import numpy as np
import statsmodels.api as sm
#import statsmodels.formula.api as smf
import random
from sklearn.linear_model import LinearRegression
import math


# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
%matplotlib inline

plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
plt.rcParams['figure.figsize'] = (10, 5)

In [6]:
from sklearn.preprocessing import StandardScaler, normalize, MinMaxScaler

In [9]:
#download_file("http://www-bcf.usc.edu/~gareth/ISL/Advertising.csv")
adv_df = pd.read_csv('http://www-bcf.usc.edu/~gareth/ISL/Advertising.csv', usecols=[1,2,3,4])
adv_df.head()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


#### 1. У нас нет NaN значений и все измерения представлены в float64.  
#### 2. Нормализацию данных не делаем, тк линейная регрессия этого не требует

In [12]:
adv_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 4 columns):
TV           200 non-null float64
radio        200 non-null float64
newspaper    200 non-null float64
sales        200 non-null float64
dtypes: float64(4)
memory usage: 6.3 KB


In [52]:
adv_df['log_tv'] = adv_df.TV.apply(lambda x: math.log(x, 2))

__________________
__________________
## Разделим датасет на трейн и тест в отношение 50:50

In [113]:
from sklearn.model_selection import train_test_split

adv_train, adv_test = train_test_split(adv_df, test_size=0.5, random_state=42, shuffle=True)

print("Total transactions in train dataset: ", len(adv_train))
print("Total transactions in test dataset: ", len(adv_test))

Total transactions in train dataset:  100
Total transactions in test dataset:  100


In [54]:
adv_train.head()

,TV,radio,newspaper,sales,log_tv
4,180.8,10.8,58.4,12.9,7.498251
32,97.2,1.5,30.0,9.6,6.602884
142,220.5,33.2,37.9,20.1,7.784635
145,140.3,1.9,9.0,10.3,7.132371
109,255.4,26.9,5.5,19.8,7.996615


### Определим фичи и таргет переменную для каждого варианта
1. sales ~ log_tv + radio 
2. sales ~ TV + radio 
3. sales ~ TV + radio + newspaper

In [93]:
x1_train = adv_train.filter(items = ['log_tv', 'radio'])
x2_train = adv_train.filter(items = ['TV', 'radio'])
x3_train = adv_train.filter(items = ['TV', 'radio','newspaper'])

y_train = adv_train['sales']

In [58]:
x1.head()

,log_tv,radio
4,7.498251,10.8
32,6.602884,1.5
142,7.784635,33.2
145,7.132371,1.9
109,7.996615,26.9


### Обучим модели с разными вариантами фичей

In [94]:
#three_x_lm = sm.OLS.from_formula("sales ~ log_tv + radio", adv_train)
#расчет метрик
# print("RSS:", np.sum(skm.resid ** 2))
# print("RSE:", np.sqrt(np.sum(skm.resid ** 2)) / (adv_train.shape[0] - 2 - 1))
# print("R^2:", skm.rsquared)


skm1 = lm.LinearRegression()
# calculate parameters
skm1.fit(x1_train, y_train)
# show them
skm1.intercept_, skm1.coef_


(-7.569208411242698, array([2.49862432, 0.21023826]))

In [95]:
skm2 = lm.LinearRegression()
# calculate parameters
skm2.fit(x2_train, y_train)
# show them
skm2.intercept_, skm2.coef_

(2.846604985488076, array([0.04379153, 0.20613395]))

In [96]:
skm3 = lm.LinearRegression()
# calculate parameters
skm3.fit(x3_train, y_train)
# show them
skm3.intercept_, skm3.coef_

(2.579115525652737, array([0.04359498, 0.19927632, 0.0146631 ]))

### Посчитаем R^2 для train

In [105]:
print('Для sales ~ log_tv + radio, R^2 train = ', skm1.score(x1_train, y_train, sample_weight=None))
print('Для sales ~ TV + radio, R^2 train = ', skm2.score(x2_train, y_train, sample_weight=None))
print('Для sales ~ TV + radio + newspaper, R^2 train = ', skm3.score(x3_train, y_train, sample_weight=None))

Для sales ~ log_tv + radio, R^2 train =  0.8997594187070235
Для sales ~ TV + radio, R^2 train =  0.9020506014720118
Для sales ~ TV + radio + newspaper, R^2 train =  0.9042613648908894


### Посчитаем R^2 для test

In [109]:
print('Для sales ~ log_tv + radio, R^2 train {:.7f}'.format(
skm1.score(adv_test[['log_tv','radio']], adv_test['sales'])))

print('Для sales ~ TV + radio, R^2 train {:.7f}'.format(
skm2.score(adv_test[['TV','radio']], adv_test['sales'])))

print('Для sales ~ TV + radio, R^2 train {:.7f}'.format(
skm3.score(adv_test[['TV','radio','newspaper']], adv_test['sales'])))


Для sales ~ log_tv + radio, R^2 train 0.9017771
Для sales ~ TV + radio, R^2 train 0.8826436
Для sales ~ TV + radio, R^2 train 0.8721005


__________________
__________________
## Проделаем это же для датафрейма с разделением на обучение и валидацию в пропорции 70:30

In [112]:
from sklearn.model_selection import train_test_split

adv_train7030, adv_test7030 = train_test_split(adv_df, test_size=0.3, random_state=42, shuffle=True)

print("Total transactions in train dataset: ", len(adv_train7030))
print("Total transactions in test dataset: ", len(adv_test7030))

Total transactions in train dataset:  140
Total transactions in test dataset:  60


In [117]:
x1_train_7030 = adv_train7030.filter(items = ['log_tv', 'radio'])
x2_train_7030 = adv_train7030.filter(items = ['TV', 'radio'])
x3_train_7030 = adv_train7030.filter(items = ['TV', 'radio','newspaper'])

y_train_7030 = adv_train7030['sales']

### Обучим модели с разными вариантами фичей и посчитаем коэффициенты b0, b1...bn

In [118]:
skm17030 = lm.LinearRegression()
# calculate parameters
skm17030.fit(x1_train_7030, y_train_7030)
# show them
skm17030.intercept_, skm17030.coef_

(-8.526874427227629, array([2.63634328, 0.20720484]))

In [121]:
skm27030 = lm.LinearRegression()
# calculate parameters
skm27030.fit(x2_train_7030, y_train_7030)
# show them
skm27030.intercept_, skm27030.coef_

(2.8376172369051815, array([0.04407736, 0.20260566]))

In [122]:
skm37030 = lm.LinearRegression()
# calculate parameters
skm37030.fit(x3_train_7030, y_train_7030)
# show them
skm37030.intercept_, skm37030.coef_

(2.70894909251591, array([0.04405928, 0.1992875 , 0.00688245]))

### Посчитаем R^2 для train

In [124]:
print('Для sales ~ log_tv + radio, R^2 train = ', skm17030.score(x1_train_7030, y_train_7030, sample_weight=None))
print('Для sales ~ TV + radio, R^2 train = ', skm27030.score(x2_train_7030, y_train_7030, sample_weight=None))
print('Для sales ~ TV + radio + newspaper, R^2 train = ', skm37030.score(x3_train_7030, y_train_7030, 
                                                                         sample_weight=None))

Для sales ~ log_tv + radio, R^2 train =  0.8993482542237392
Для sales ~ TV + radio, R^2 train =  0.9048377867980043
Для sales ~ TV + radio + newspaper, R^2 train =  0.9055159502227753


### Посчитаем R^2 для test

In [125]:
print('Для sales ~ log_tv + radio, R^2 train {:.7f}'.format(
skm17030.score(adv_test7030[['log_tv','radio']], adv_test7030['sales'])))

print('Для sales ~ TV + radio, R^2 train {:.7f}'.format(
skm27030.score(adv_test7030[['TV','radio']], adv_test7030['sales'])))

print('Для sales ~ TV + radio, R^2 train {:.7f}'.format(
skm37030.score(adv_test7030[['TV','radio','newspaper']], adv_test7030['sales'])))


Для sales ~ log_tv + radio, R^2 train 0.9143971
Для sales ~ TV + radio, R^2 train 0.8656254
Для sales ~ TV + radio, R^2 train 0.8609467


__________________
__________________
## Проделаем это же для датафрейма с разделением на обучение и валидацию в пропорции 80:20

In [126]:
from sklearn.model_selection import train_test_split

adv_train8020, adv_test8020 = train_test_split(adv_df, test_size=0.2, random_state=42, shuffle=True)

print("Total transactions in train dataset: ", len(adv_train8020))
print("Total transactions in test dataset: ", len(adv_test8020))

Total transactions in train dataset:  160
Total transactions in test dataset:  40


In [128]:
x1_train_8020 = adv_train8020.filter(items = ['log_tv', 'radio'])
x2_train_8020 = adv_train8020.filter(items = ['TV', 'radio'])
x3_train_8020 = adv_train8020.filter(items = ['TV', 'radio','newspaper'])

y_train_8020 = adv_train8020['sales']

### Обучим модели с разными вариантами фичей и посчитаем коэффициенты b0, b1...bn

In [129]:
skm18020 = lm.LinearRegression()
# calculate parameters
skm18020.fit(x1_train_8020, y_train_8020)
# show them
skm18020.intercept_, skm18020.coef_

(-8.83303046475564, array([2.68456332, 0.20432743]))

In [130]:
skm28020 = lm.LinearRegression()
# calculate parameters
skm28020.fit(x2_train_8020, y_train_8020)
# show them
skm28020.intercept_, skm28020.coef_

(3.0282552507833014, array([0.0447283 , 0.19066726]))

In [131]:
skm38020 = lm.LinearRegression()
# calculate parameters
skm38020.fit(x3_train_8020, y_train_8020)
# show them
skm38020.intercept_, skm38020.coef_

(2.979067338122629, array([0.04472952, 0.18919505, 0.00276111]))

### Посчитаем R^2 для train

In [133]:
print('Для sales ~ log_tv + radio, R^2 train = ', skm18020.score(x1_train_8020, y_train_8020, sample_weight=None))
print('Для sales ~ TV + radio, R^2 train = ', skm28020.score(x2_train_8020, y_train_8020, sample_weight=None))
print('Для sales ~ TV + radio + newspaper, R^2 train = ', skm38020.score(x3_train_8020, y_train_8020, 
                                                                         sample_weight=None))

Для sales ~ log_tv + radio, R^2 train =  0.9011051213818659
Для sales ~ TV + radio, R^2 train =  0.8955982149747163
Для sales ~ TV + radio + newspaper, R^2 train =  0.8957008271017818


### Посчитаем R^2 для test

In [134]:
print('Для sales ~ log_tv + radio, R^2 train {:.7f}'.format(
skm18020.score(adv_test8020[['log_tv','radio']], adv_test8020['sales'])))

print('Для sales ~ TV + radio, R^2 train {:.7f}'.format(
skm28020.score(adv_test8020[['TV','radio']], adv_test8020['sales'])))

print('Для sales ~ TV + radio, R^2 train {:.7f}'.format(
skm38020.score(adv_test8020[['TV','radio','newspaper']], adv_test8020['sales'])))


Для sales ~ log_tv + radio, R^2 train 0.9235658
Для sales ~ TV + radio, R^2 train 0.9005833
Для sales ~ TV + radio, R^2 train 0.8994380


### Вывод:
### Максимальную долю объясненной дисперсии R^2 мы получили, разбив множество на трейн и тест в пропорции 80:20.
### Лучшая модель: sales ~ log_tv + radio
### R^2 = 0.9235658